In [1]:
# Most basic stuff for EDA.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Core packages for text processing.

import string
import re

# Libraries for text preprocessing.

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Loading some sklearn packaces for modelling.

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics import f1_score, accuracy_score

# Some packages for word clouds and NER.

from wordcloud import WordCloud, STOPWORDS
from collections import Counter, defaultdict
from PIL import Image
import spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz
import en_core_web_sm

# Core packages for general use throughout the notebook.

import random
import warnings
import time
import datetime

# For customizing our plots.

from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

# Loading pytorch packages.

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

# Setting some options for general use.

stop = set(stopwords.words('english'))
plt.style.use('fivethirtyeight')
sns.set(font_scale=1.5)
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
warnings.filterwarnings('ignore')


#Setting seeds for consistent results.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\mucle\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:873: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.0/12.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011737 sha256=73ebfce61bdcb6683f600f29b209a91ee0828bc5117bd177c5f31b2cfa31de07
  Stored in directory: c:\users\mucle\appdata\local\pip\cache\wheels\57\07\ed\22b6eecc1314cf0e1cbfb45ebeb5ba063dfbc86645f6f23c23
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.4.0
    Uninstalling en-core-web-sm-3.4.0:
      Successfully uninstalled en-core-web-sm-3.4.0


In [197]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
# Loading NER.
nlp = en_core_web_sm.load() 

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)
  Using cached setuptools-65.3.0-py3-none-any.whl (1.2 MB)


ERROR: To modify pip, please run the following command:
D:\anaconda3\python.exe -m pip install -U pip setuptools wheel


Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 12.8/12.8 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
data = pd.read_csv('training_data.csv', on_bad_lines='skip')
datassss=data[0:10000]

In [3]:
datass=data[0:10001]
datass

,tweet_text,lang,timestamp,lat,lon
0,Nak berak masa berlari is another level of ann...,in,2021-08-24T00:20:01.000Z,3.14853065,101.705305
1,@cricketaakash @vikrantgupta73 India never ha...,en,2021-08-19T07:27:34.000Z,30.7303855,76.7765244
2,,NaN,NaN,NaN,NaN
3,"Just posted a photo @ Cocoa,Stati Uniti,en,202...",NaN,NaN,NaN,NaN
4,"كفيل في غضبي أكون اسوأ واحد..""",ar,2021-08-09T19:47:18.000Z,30.3560454,31.3315573
...,...,...,...,...,...
9996,We see you #Knights! \n\nStory: https://t.co/1...,en,2021-08-18T15:35:35.000Z,33.1946515,-96.699604
9997,"Acaba de publicar una foto en Pucón,Chile,es,2...",NaN,NaN,NaN,NaN
9998,You don’t change a machine by voting out old p...,NaN,NaN,NaN,NaN
9999,Machine will still run the same.,NaN,NaN,NaN,NaN


In [4]:
# import module
import geograpy3
from geograpy3 import extraction
from geopy.geocoders import Nominatim
sentence44 = []
sentence48 = [[]]
for index in range(datassss.shape[0]):
    #Latitude = datassss.iloc[index,3].astype(float)
    #Longitude = datassss.iloc[index,4].astype(float)
    #link = 'http://www.bbc.com/news/world-europe-26919928'
    #e = extraction.Extractor(datassss["tweet_text"].to_string())
    #e.find_entities()

    # You can now access all of the places found by the Extractor
    
    #places = geograpy3.get_place_context(url = link)
    text_input = data.iloc[index,:1].to_string()
    places = geograpy3.get_place_context(text = text_input)
    text_input = datassss.iloc[index,:1].to_string()
    more_places = geograpy3.get_place_context(text = text_input)
    sentence48.append(more_places.places)
    print(more_places.places)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mucle\AppData\Roaming\nltk_data...
[nltk_data]   Package avera

[]
['India']
[]
['Just', 'Stati Uniti']
['كفيل']
['Kagurazaka', 'Japon']
['GoldbergVariat2 Yo']
['TIKTOK', 'FB']
['U.S.']
['Gespräch']
[]
['SENER_mx Buenas']
['英会話']
['BSNLCorporate', 'BSNL']
[]
['India']
['武庫川駅', '尼崎市', 'Japon']
['김호중']
['金浩仲', 'キムホジュン']
['트바로티김호중', '노래하는사람_김호중']
['김호중_고맙소_감사의메세지']
['김호중_광복절콘서트']
['광복절', '나라사랑']
['김호중이장르다']
[]
[]
[]
[]
['Tumbuhkan Jiwa Peduli', 'Berbagi', 'Indonesia']
[]
['Kirksville', 'United States']
['تیرے']
[]
[]
[]
[]
['Just', 'United States']
['CONGRATULATIONS']
['San Martín']
[]
[]
['Caracas']
[]
['Barca']
['Just', 'Santa Leticia Hotel']
[]
[]
[]
[]
['Congratulations']
[]
['Severe Thunderstorm Warning', 'Linc']
[]
['United States']
['Severe Thunderstorm Warning', 'Luca']
['Just', 'Regno Unito']
['TrafficAlert']
[]
['Santiago', 'Cali']
['United States']
['Kalesi']
[]
[]
[]
[]
['Gespräch']
['Alarm', 'Source', 'Toronto']
[]
[]
[]
[]
[]
['ハナヤマZ', '田原本町', 'Japon']
['Colombia']
[]
['Prancis']
[]
[]
['Ujjain', 'Pakistan']
['Los', 'Serie']
['おうちごはん']
[

[]
['Just']
['Vacina']
[]
['Bila']
[]
[]
[]
['São João Batista']
[]
[]
['Brésil']
[]
[]
['Just', 'Putrajaya', 'Malaysia']
['Medical', 'Toronto']
['Forget City', 'Nigeria']
[]
[]
['สนใจโทร']
['Deutschland', 'Impfstoff']
['Severe Thunderstorm Warning', 'Bois']
['健康って大事']
['麦茶割り']
['野菜炒め']
[]
['Miles Street', 'Stati Uniti']
[]
[]
['半﨑美子']
[]
['CARALHO']
[]
['Bay', 'Watch']
['Self Made', 'Riyadh']
['Le Château']
[]
['刚刚发布了视频', 'China']
['WalkingBoston', 'Boston']
['Los Angeles']
[]
[]
['Argentina']
[]
['HACKER']
[]
[]
[]
['Pandemie', 'Impfu']
['HAHAHAHAHAHAHAH']
[]
['Just', 'Basin', 'United States']
[]
['Don Draper']
['Just', 'Brooklyn']
[]
[]
[]
['Sibirien']
['EldrigeSupan11']
[]
['ブレイク植田', 'Iwaki', 'Japan']
[]
[]
['吉野家', '尼宝線池尻店', '伊丹市', 'Ja']
['Imaginate', 'Afgano']
['Belfast', 'Re']
['うぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅぅ']
[]
['Karlsruhe']
['متذكرين']
['Coe Felipe']
['Just', 'New York', 'United States']
[]
[]
['General Caballero']
['IntermediaAPF', 'IntermediaInternacion']
[]
['LO

['麺や', '六等星', '川崎市多摩区', 'Japon']
[]
['JetBlue']
['Just']
['BirminghamCity', 'AFC Bournemouth', 'MatchPrevi']
[]
['DVISD_PES', 'Popham Math Teachers']
[]
[]
['Melbourne']
[]
[]
['Kang Botswan']
['LifeSavior_GodKabir']
['Gabriel08cardos De']
['恵庭市']
['Just', 'Gresham', 'United States']
['スキーヤー']
[]
[]
[]
['Deixem', 'Marinho', 'Baixa']
[]
['Delayed Report', 'SSW Summerdale']
[]
['Just', 'Beavercreek', 'Stati Uniti']
[]
[]
['神奈川県']
[]
[]
['山口県', '宇部市', 'フィットネス']
['筋トレ', '食事指導', '運動は運を動かす']
['パーソナルトレーニング', 'パーソナルトレーナー']
[]
[]
[]
[]
[]
['Katanya']
['Guildford']
[]
['HilareeBanks', 'Florida', 'United States']
[]
[]
['Visited']
[]
['Whaler', 'Carmel']
['Serio', 'Brésil']
[]
['Zefu__anime']
['Acabou', 'Belo Horizonte']
[]
[]
[]
['SandeepDadarMNS']
[]
['Just', 'Sekolah Pascasarjana UIN']
['نقل']
['Low Carb']
[]
['Rat Trap', 'Kansas City', 'United States']
['Dirgahayu Republik Indonesia']
['HUTRI76']
[]
[]
[]
[]
['Bukit Mertajam', 'Pulau', 'Pinang']
['JetBlue']
[]
[]
['Pakistan']
[]
['Manche Tage

['México']
[]
['Lmfao']
['Carpool', 'Munich']
['India', 'Afgh']
['Italy', 'Covid', 'Delta']
[]
['駿河屋札幌ノルベサ店', '札幌市', 'Japon']
[]
[]
['Just', 'Beaverton', 'United States']
[]
['Phish Ascend Nashville']
[]
['Mexico City', 'Mexi']
[]
[]
[]
[]
['Vitinho']
['Renato']
['Qual', 'Vanderson']
['Rafinha', 'Deixa']
[]
[]
['USA']
['姫路駅', '姫路市', 'Japon']
[]
[]
[]
[]
[]
['緊急ピットイン', 'セルフ壬生']
[]
['Rehab Support Services']
[]
['Run4India']
['Tesla', 'WOC']
['Chile']
['Acabou', 'Rio', 'Janeiro']
[]
[]
[]
[]
[]
[]
[]
['Someone', 'Sac', 'California']
['Joseph_C_Rocha']
['帰宅ラッシュにハマる', '西白河郡矢吹町', 'Japon']
['Monterrey', 'Mexic']
[]
[]
['Kabul', 'Militärf']
['Just']
[]
[]
['Только']
['잔머리']
[]
['garedVS']
['島田市', 'Japan']
[]
['Brampton']
['力丸家', 'Fukuroi', 'Giappone']
[]
[]
[]
[]
[]
[]
['Cuba', 'Brasil']
[]
['Just', 'United Stat']
[]
['Fillét']
['Columns', 'Glen Ellyn']
[]
['Turki_alalshikh']
['Turki_alalshikh']
['الخيل']
[]
['Just', 'Los Angeles']
[]
[]
['List']
[]
[]
[]
['San Juan']
['Carpool', 'Naples', 'S

...

['Nunca']
[]
['MPH']
[]
[]
[]
['Palestine']
['Cremorne']
['Repss Campeche', 'Campeche']
['TS Fred']
[]
[]
[]
['San Jose', 'Santa Clara', 'Unit']
['Kabul', 'Militärf']
['GrahamReid']
[]
['ไมดด']
[]
[]
[]
['Parece']
['Maurice']
['الإمام']
['الحجة']
[]
[]
[]
[]
[]
['u41mvT']
[]
['Шопска']
[]
['Afghanistan']
['Malesia']
[]
[]
['WangYibo王一博']
['ダイユーエイト', '白石市', 'Japan']
[]
['اذا']
[]
[]
[]
['NATO']
['Maria_369_']
[]
[]
[]
['Paris_neyoo']
[]
[]
[]
['Gujrat']
['Sibirien']
[]
[]
['Acabou', 'Floriano', 'Brazi']
['Indonesia']
[]
[]
['Main']
['KarlinhaB7 Estou']
[]
[]
['US9', 'North']
[]
[]
[]
['Brasil']
['COVID']
[]
[]
[]
['Santiago Accidente']
[]
['TERRY']
['Just']
['Por']
['下北沢駅', '世田谷区', 'Japan']
[]
['Wait']
[]
[]
['プレミアムロールケーキ食べたい', 'を投稿すると無']
['Bogotá', 'Colombia']
['BE']
[]
[]
[]
[]
['Assistindo']
['E411', 'Luxembourg', 'Rechter']
[]
[]
['Vegas']
[]
[]
['Por']
['Señores']
[]
[]
['METROCALI Muy']
['Hawaii']
['Sibirien']
['Adoro BOLAS', 'Cuidado']
['鳥取駅', '鳥取', 'Japan']
['Tom N Toms Coffee']

['Sorry Doc']
[]
['Barış']
[]
[]
['Confirmado', 'Stati Uniti']
[]
[]
['Peru']
['RB9 Psje']
['港区', 'Japan']
['Estaremos']
[]
[]
['USOpen', 'NYC']
[]
['Virgen']
[]
[]
[]
['Kabul', 'Militärf']
['كملت', 'أنجار']
['Myanmar']
['Bona']
['Dirgahayu Republik']
['Sioux Falls']
[]
['ANDHA']
[]
[]
[]
['Genel Başkanımız Sayın']
['JetBlue']
['Kabul', 'Militärf']
['São Paulo']
[]
['Comienza', 'San Carlos']
['Kabul', 'Militärf']
[]
[]
[]
['Parag']
['Pere']
[]
['Just']
[]
[]
[]
['刚刚发了照片']
[]
[]
['MainoHustleHard']
['Belo Horizonte', 'VotoAuditavelJa']
['BMRDA', 'ForSale']
[]
[]
['MAÑANERO']
['Jake Stimple', 'Oklahoma']
['Pandemie', 'Impfu']
[]
[]
['Balai Bom']
[]
[]
[]
[]
[]
['Rio']
['AlieHaleyy']
['Florida']
['Aya Yorgi Koyu', 'Çeşme', 'Turquie']
[]
['FREESTYLE', 'Animas']
[]
['Scott', 'Bar-B-Que']
[]
['Ωωωω']
[]
['Piscinitas']
[]
[]
['America']
['แหกหมด']
[]
[]
['Sibirien']
[]
['Steam Locomotive']
['JetBlue']
['Debería']
[]
['Escondido', 'Stati Uniti']
[]
[]
['Turkey']
['Parliament Street', 'Dundas S

['築地本願寺カフェ Tsumugi', '中央区', 'Japon']
[]
['Kabul', 'Militärf']
['Bonnie']
['Nebraska']
[]
[]
['Illinois']
['España']
[]
[]
['NY29A', 'CR']
[]
['FuN']
[]
['남여노소', '빅박스', '타미야', '자존감']
[]
['Kaplan']
[]
['Seattle']
[]
[]
[]
[]
['Café']
['Mérida La']
['Texas Chicken', 'Nilai', 'Mala']
[]
[]
[]
['Hawaii']
[]
[]
[]
[]
[]
[]
['StephXAGs Colorado', 'United States']
[]
[]
['Houston']
[]
['USGS', 'Utah']
['Bapak Menteri Pertanian RI']
['ZeeBusiness']
['Brasil', 'Tokyo']
['Kok Majid Ke Pulau Teluk Ranj']
['Just', 'Hudson River Park']
['今週いっぱいはまりやさんのSEPTEMBERとEarthのSEPTEMBERを聴くことになり']
['Agora', 'Luciano']
['Andalucia', 'Spain']
['Dios']
[]
['Freedom']
['Caracas', 'Venezue']
[]
['Sibirien']
[]
[]
[]
['Kabul', 'Militärf']
[]
['Chott', 'Jerid', 'Tunisie']
['ZAP']
[]
['Ang']
[]
[]
[]
['Afghanis']
[]
['Sekarang']
[]
['Christian Campos', 'Alisal']
['VImvinit007']
['Jitpur']
['EAMaddenNFL', 'Madden22 PS4']
['TimCurran8']
[]
[]
['Indonesia']
[]
[]
[]
[]
['沖縄尚学']
[]
['県勢春夏100勝目']
[]
[]
[]
[]
[]
['Mérida Yuc

[]
['MAÑANA', 'BTW']
['Roma', 'Italie']
['這座中國風城門是台北的北門', '與台北府城一同建於1884年', '光緒10年', '是目前台北唯一並未改']
['Caroli']
['Okanagan']
[]
['Total']
['Bloody Blair']
[]
[]
['LORD']
[]
['Calmness', 'Guaymas', 'Mexique']
['Ireland']
[]
[]
['Just', 'Lagos', 'Nigeria']
['Caracas']
[]
[]
[]
['PorscheZula4']
[]
['Acabou', 'São Francisco']
[]
['NYC']
[]
[]
[]
[]
['Otros']
['Portugal', 'STF']
['安城市', 'Japa']
['Acabou', 'Jacaraípe', 'Braz']
['Colombia']
[]
['Acabou', 'Brasil', 'País']
[]
[]
[]
['AMOR DE', 'SOLA', 'LGBTQ']
[]
[]
['Trump']
['深圳兼职', '深圳约炮', '深圳喝茶', '个人兼职']
[]
[]
[]
['Pablo']
[]
['Miami']
['Just', 'Kelowna', 'Canada']
[]
['上石神井駅', '練馬区', 'Japon']
['Deutschland', 'Impfstoff']
[]
['Esto']
[]
['Backpacks', 'Laptop', 'CARGO']
[]
['Puebla']
[]
['Just', 'Los Angeles']
[]
[]
[]
[]
['Indonesia', 'Hopefully']
['Anantara Phuket Villas', 'Thalang', 'Thaïl']
[]
[]
['Kerry James Marshall', 'Paris']
[]
['Comunidad', 'Madrid']
['Kabul', 'Militärf']
[]
[]
['Marseille', 'Cette']
['Hamburg', 'Fest']
['Bogota']
[

['Just']
[]
[]
['TORNADO']
[]
[]
[]
['Vale', 'Aproveite']
[]
[]
['Mllesoss0']
['Zon']
[]
['Prospect Drugs', 'Brooklyn']
['Ong']
['Abyss B']
['نعس']
[]
[]
['Period']
['Just']
[]
['Bale']
['East Boumd', 'Louisville', 'Et']
[]
[]
[]
[]
['باريس']
[]
['URGENTE', 'Brésil']
['Result', 'OldNote']
['Pty Ltd']
['Regno Unito']
[]
['Centro', 'Memphis']
['Barcelona', 'RealSociedad']
[]
[]
[]
['Leland', 'United St']
[]
['inBLOOM']
[]
[]
[]
['Afghanistan', 'Somalia', 'Libya']
[]
[]
[]
[]
[]
['Happy National Dog']
['LORD']
[]
['JetBlue']
[]
[]
[]
[]
[]
['Canine Trio', 'Max', 'Owl']
['Parlazaaaaaaaaa']
[]
['牧野真莉']
[]
['Ntungamo', 'Isingiro']
['Just']
[]
['Kabul', 'Militärf']
[]
['Msm']
['Parliament']
['白石区複合庁舎', '札幌市', 'Giappone']
['Bucky']
['都ホテル京都八条', 'Kyoto', 'Japon']
['Inés Perado']
['Campo']
['Idaho']
['South']
[]
[]
[]
['Kabul', 'Militärf']
['Настроение']
[]
[]
[]
[]
['豊橋神野埠頭', '豊橋市', 'Giappone']
[]
[]
[]
['Essa Alice Portugal']
['HomeboyzRadio Githima']
['India']
['Siappp Merapat']
['Como', 'Arg

In [5]:
df = pd.DataFrame(sentence48)
df

,0,1,2,3,4,5,6,7
0,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None
2,India,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None
4,Just,Stati Uniti,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
9996,Van,None,None,None,None,None,None,None
9997,None,None,None,None,None,None,None,None
9998,Pucón,Chile,None,None,None,None,None,None
9999,None,None,None,None,None,None,None,None


In [6]:
# render to coordinates


import pandas as pd


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="spec1")

from geopy.extra.rate_limiter import RateLimiter

from tqdm import tqdm
tqdm.pandas()

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['location0'] = df.iloc[:,0].progress_apply(geocode)
df['location1'] = df.iloc[:,1].progress_apply(geocode)
df['location2'] = df.iloc[:,2].progress_apply(geocode)

 64%|██████████████████████████████████████████████▋                          | 6402/10001 [9:51:21<1:02:08,  1.04s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*(None,), **{}).
Traceback (most recent call last):
  File "C:\Users\mucle\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\mucle\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "D:\anaconda3\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "D:\anaconda3\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "D:\anaconda3\lib\http\client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "D:\anaconda3\lib\socket.py", line 704, in readinto
    return self._s

RateLimiter caught an error, retrying (1/2 tries). Called with (*('广州天河高端',), **{}).
Traceback (most recent call last):
  File "C:\Users\mucle\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\mucle\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "D:\anaconda3\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "D:\anaconda3\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "D:\anaconda3\lib\http\client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "D:\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "D:\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  F

100%|██████████████████████████████████████████████████| 10001/10001 [2:48:00<00:00,  1.01s/it]


In [25]:

df['point0'] = df['location0'].apply(lambda loc: tuple(loc.point) if loc else None)
df['point1'] = df['location1'].apply(lambda loc: tuple(loc.point) if loc else None)
df['point2'] = df['location2'].apply(lambda loc: tuple(loc.point) if loc else None)

In [26]:
df

,0,1,2,3,4,5,6,7,location0,location1,location2,point0,point1,point2
0,None,None,None,None,None,None,None,None,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
1,None,None,None,None,None,None,None,None,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
2,India,None,None,None,None,None,None,None,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
3,None,None,None,None,None,None,None,None,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
4,Just,Stati Uniti,None,None,None,None,None,None,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,None,None,None,None,None,None,None,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
9997,None,None,None,None,None,None,None,None,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
9998,Pucón,Chile,None,None,None,None,None,None,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)"
9999,None,None,None,None,None,None,None,None,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"


In [27]:
k = ['lat','lon'] 
datass[k] = datass[k].apply(pd.to_numeric, errors = 'coerce').dropna() # data type casting and cleaning

In [28]:
df1 = df
ds = datassss

In [29]:
ds = datass

In [30]:
k = ['lat','lon'] 
datassss[k] = datassss[k].apply(pd.to_numeric, errors = 'coerce').dropna() # data type casting and cleaning

In [31]:

df1 = df.fillna(value=np.nan)
df1 = df1.fillna(0)
df1

,0,1,2,3,4,5,6,7,location0,location1,location2,point0,point1,point2
0,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
1,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
2,India,0,0,0,0,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
3,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
4,Just,Stati Uniti,0,0,0,0,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,0,0,0,0,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
9997,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
9998,Pucón,Chile,0,0,0,0,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)"
9999,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"


In [32]:
ds = ds.fillna(value=np.nan)
ds = ds.fillna(0)
ds

,tweet_text,lang,timestamp,lat,lon
0,Nak berak masa berlari is another level of ann...,in,2021-08-24T00:20:01.000Z,3.148531,101.705305
1,@cricketaakash @vikrantgupta73 India never ha...,en,2021-08-19T07:27:34.000Z,30.730386,76.776524
2,,0,0,0.000000,0.000000
3,"Just posted a photo @ Cocoa,Stati Uniti,en,202...",0,0,0.000000,0.000000
4,"كفيل في غضبي أكون اسوأ واحد..""",ar,2021-08-09T19:47:18.000Z,30.356045,31.331557
...,...,...,...,...,...
9996,We see you #Knights! \n\nStory: https://t.co/1...,en,2021-08-18T15:35:35.000Z,33.194651,-96.699604
9997,"Acaba de publicar una foto en Pucón,Chile,es,2...",0,0,0.000000,0.000000
9998,You don’t change a machine by voting out old p...,0,0,0.000000,0.000000
9999,Machine will still run the same.,0,0,0.000000,0.000000


In [33]:
df1

,0,1,2,3,4,5,6,7,location0,location1,location2,point0,point1,point2
0,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
1,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
2,India,0,0,0,0,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
3,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
4,Just,Stati Uniti,0,0,0,0,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,0,0,0,0,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
9997,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"
9998,Pucón,Chile,0,0,0,0,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)"
9999,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)"


In [34]:
df1.shape

(10001, 14)

In [35]:
df1.iloc[index,13]

(44.933143, 7.540121, 0.0)

In [36]:
# Geopy distance module is first imported for computation 
from geopy.distance import geodesic as GD
# Latitude+","+Longitude Next, input the latitude and longitude data for Nairobi and Cairo.  
sentence = []
for index in range(df1.shape[0]):
    #Latitude1=datassss.iloc[index,3].astype(str)
    #Longitude1 = datassss.iloc[index,4].astype(str)
    X1 = [ds.iloc[index,3].astype(str)+","+ds.iloc[index,4].astype(str)]  
    X2=df1.iloc[index,11]
    #C=df.iloc[96,9]
    X3=df1.iloc[index,12]
    X4 =df1.iloc[index,13] 
    
    if df1.iloc[index,11]!= 0 and type(df1.iloc[index,11]) is tuple:       
    # Finally, print the distance between the two locations in kilometers. 
        print("The distance between X1 and X2 is :",GD(X1,X2).m)
        df1['dist1'] = GD(X1,X2).m
    elif df1.iloc[index,12]!= 0 and type(df1.iloc[index,12]) is tuple :
        print("The distance between X1 and X3 is :",GD(X1,X3).m)
        df1['dist2'] = GD(X1,X3).m
    elif df1.iloc[index,13]!= 0 and type(df1.iloc[index,13]) is tuple  :
        print("The distance between X1 and X4 is :",GD(X1,X4).m) 
        df1['dist3'] = GD(X1,X4).m

The distance between X1 and X2 is : 10092994.91195653
The distance between X1 and X2 is : 6104568.48273513
The distance between X1 and X2 is : 8850082.942465011
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 2330364.803701886
The distance between X1 and X2 is : 6265812.820248931
The distance between X1 and X2 is : 14276487.094952038
The distance between X1 and X3 is : 9179796.849587178
The distance between X1 and X2 is : 9004684.143314315
The distance between X1 and X2 is : 5463740.893068639
The distance between X1 and X2 is : 8472870.639656566
The distance between X1 and X2 is : 9938977.34740713
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X2 is : 4770429.888311475
The distance between X1 and X3 is : 14483827.548576294
The distance between X1 and X2 is : 6597923.29096117
The distance between X1 and X2 is : 8850082.942465011
The distance between X1 and X2 is : 14000789.655700142
The distance between X1 and X3

The distance between X1 and X3 is : 5997997.689945321
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 5751388.011416578
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X2 is : 5041790.626581338
The distance between X1 and X2 is : 8252791.0059812
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 9787000.681379095
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X3 is : 9373019.006902182
The distance between X1 and X2 is : 6392859.971532237
The distance between X1 and X2 is : 8572967.515193596
The distance between X1 and X3 is : 4331

The distance between X1 and X2 is : 9854240.531771671
The distance between X1 and X2 is : 9445338.260598432
The distance between X1 and X2 is : 7262687.712237669
The distance between X1 and X2 is : 8765995.56426521
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 6692699.847966493
The distance between X1 and X2 is : 9029929.366885548
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9215753.330151746
The distance between X1 and X2 is : 11268377.043460151
The distance between X1 and X2 is : 8859596.468224457
The distance between X1 and X2 is : 10346782.976556335
The distance between X1 and X2 is : 9274651.667686198
The distance between X1 and X2 is : 7943806.682361942
The distance between X1 and X2 is : 6392859.971532237
The distance between X1 and X2 is : 6389839.552277051
The distance between X1 and X2 is : 7663700.314977073
The distance between X1 and X2 is : 6572644.709249645
The distance between X1 and X

The distance between X1 and X2 is : 9693207.229318812
The distance between X1 and X2 is : 12562484.82246195
The distance between X1 and X3 is : 8711958.107830768
The distance between X1 and X2 is : 6165443.389114686
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9877857.04262644
The distance between X1 and X2 is : 9371366.999128615
The distance between X1 and X2 is : 5547960.648945619
The distance between X1 and X2 is : 6572644.709249645
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 7516822.589864234
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5836073.424335459
The distance between X1 and X2 is : 7230549.335440681
The distance between X1 and X2 is : 6487253.901363957
The distance between X1 and X2 is : 6532820.984757907
The distance between X1 and X2 is 

The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X3 is : 13457354.077772507
The distance between X1 and X2 is : 9373019.006902182
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 7305815.779089264
The distance between X1 and X2 is : 9630963.951888153
The distance between X1 and X2 is : 7235007.6809084425
The distance between X1 and X2 is : 12071604.490685236
The distance between X1 and X2 is : 1296034.3683601846
The distance between X1 and X2 is : 6363193.104347413
The distance between X1 and X2 is : 6268215.738815309
The distance between X1 and X3 is : 7269450.295146299
The distance between X1 and X2 is : 9639802.917130033
The distance between X1 and X2 is : 8499643.695196584
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 13650201.572396949
The distance between X1 and 

...

The distance between X1 and X2 is : 9720387.830166945
The distance between X1 and X2 is : 8041880.550138667
The distance between X1 and X2 is : 8970735.69397161
The distance between X1 and X2 is : 1234076.1530011161
The distance between X1 and X2 is : 9748671.5089944
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 8850586.811428336
The distance between X1 and X2 is : 13777387.789196393
The distance between X1 and X3 is : 8744834.61791769
The distance between X1 and X2 is : 9938977.34740713
The distance between X1 and X2 is : 9692100.466588518
The distance between X1 and X2 is : 4723625.918170373
The distance between X1 and X3 is : 10676499.48697488
The distance between X1 and X2 is : 6387204.279844461
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 11619975.150415134
The distance between X1 and X2 is : 10644052.152353836
The distance between X1 and X2 i

The distance between X1 and X2 is : 12533127.84488987
The distance between X1 and X2 is : 1519799.3794016424
The distance between X1 and X2 is : 9869220.951848079
The distance between X1 and X2 is : 8547869.30483727
The distance between X1 and X2 is : 8480655.398418149
The distance between X1 and X2 is : 1537320.311287876
The distance between X1 and X2 is : 10568516.780968608
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 8547085.670216147
The distance between X1 and X2 is : 5647112.054557286
The distance between X1 and X2 is : 3656837.8404782806
The distance between X1 and X2 is : 7881095.823681175
The distance between X1 and X2 is : 15440270.880791534
The distance between X1 and X2 is : 6152292.798921277
The distance between X1 and X2 is : 9816645.816044446
The distance between X1 and X2 is : 9747560.836858
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 

The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5041790.626581338
The distance between X1 and X2 is : 9309228.528222661
The distance between X1 and X2 is : 8970735.69397161
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 3985871.2447754918
The distance between X1 and X2 is : 10370199.663055874
The distance between X1 and X2 is : 7903836.085382228
The distance between X1 and X2 is : 5415725.981283017
The distance between X1 and X2 is : 9382309.085490022
The distance between X1 and X2 is : 1769847.9209767247
The distance between X1 and X2 is : 6102430.725486123
The distance between X1 and X3 is : 1208649.381969248
The distance between X1 and X2 is : 5124829.280414605
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X2 is : 8091325.261888423
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 

The distance between X1 and X2 is : 11685290.045667488
The distance between X1 and X3 is : 5652437.330006801
The distance between X1 and X2 is : 2991334.8723793956
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 6597923.29096117
The distance between X1 and X2 is : 8115144.927548277
The distance between X1 and X2 is : 9029929.366885548
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9706922.771742452
The distance between X1 and X2 is : 7575956.193358523
The distance between X1 and X2 is : 5166446.831925396
The distance between X1 and X2 is : 11226013.736922279
The distance between X1 and X2 is : 11033315.7380009
The distance between X1 and X2 is : 9713014.284472348
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 9857220.472008085
The distance between X1 and X2 is : 9373019.006902182
The distance between X1 and X2 is : 9182847.427707085
The distance between X1 and X2

The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 10093776.180358296
The distance between X1 and X2 is : 5901630.085333386
The distance between X1 and X2 is : 13893278.253255047
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X2 is : 5997997.689945321
The distance between X1 and X2 is : 9630627.735608142
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 15474002.615909247
The distance between X1 and X2 is : 15392890.133109603
The distance between X1 and X3 is : 7817686.70578981
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X2 is : 5032884.25415879
The distance between X1 and X3 is : 9393897.556367282
The distance between X1 and X3 is : 5032884.25415879
The distance between X1 and X2 is 

In [37]:
df1

,0,1,2,3,4,5,6,7,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3
0,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
1,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
2,India,0,0,0,0,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
3,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
4,Just,Stati Uniti,0,0,0,0,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,0,0,0,0,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
9997,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
9998,Pucón,Chile,0,0,0,0,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06
9999,0,0,0,0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06


In [38]:
df2 = df1

In [39]:
df2["class_4"] = None
df2 = df2[[0,1,2,3,'location0','location1', 'location2', 'point0', 'point1', 'point2', 'dist1', 'dist2', 'dist3','class_4']]

In [40]:
df2

,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
2,India,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
4,Just,Stati Uniti,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
9997,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
9998,Pucón,Chile,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None
9999,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,None


In [41]:
df2.iloc[index,11]

5032884.25415879

In [48]:
# 1-4 range coding 1- lowest precision 4 highest precision of text - coordinate correspondance
for  index in range(df2.shape[0]):    
    if df2.iloc[index,10] < 2000000 or df2.iloc[index,11]< 2000000 or df2.iloc[index,12] < 2000000 :
        df2.iloc[index,13] = 1
    elif 2000000 < df2.iloc[index,10] < 4000000 or 2000000 < df2.iloc[index,11] < 4000000 or 2000000 < df2.iloc[index,12] < 4000000:
        df2.iloc[index,13] = 2
    elif 4000000 < df2.iloc[index,10] < 10000000 or 4000000 < df2.iloc[index,11] < 10000000 or 4000000 < df2.iloc[index,12] < 10000000:
        df2.iloc[index,13] = 3
    else:
        df2.iloc[index,13] = 4 


In [49]:
df2

,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
2,India,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
4,Just,Stati Uniti,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9997,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9998,Pucón,Chile,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9999,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3


In [ ]:
ds

In [50]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [51]:
ds.index = np.arange(1, len(ds) + 1)
ds

,tweet_text,lang,timestamp,lat,lon
1,Nak berak masa berlari is another level of ann...,in,2021-08-24T00:20:01.000Z,3.148531,101.705305
2,@cricketaakash @vikrantgupta73 India never ha...,en,2021-08-19T07:27:34.000Z,30.730386,76.776524
3,,0,0,0.000000,0.000000
4,"Just posted a photo @ Cocoa,Stati Uniti,en,202...",0,0,0.000000,0.000000
5,"كفيل في غضبي أكون اسوأ واحد..""",ar,2021-08-09T19:47:18.000Z,30.356045,31.331557
...,...,...,...,...,...
9997,We see you #Knights! \n\nStory: https://t.co/1...,en,2021-08-18T15:35:35.000Z,33.194651,-96.699604
9998,"Acaba de publicar una foto en Pucón,Chile,es,2...",0,0,0.000000,0.000000
9999,You don’t change a machine by voting out old p...,0,0,0.000000,0.000000
10000,Machine will still run the same.,0,0,0.000000,0.000000


In [52]:
df2

,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
0,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
1,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
2,India,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
3,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
4,Just,Stati Uniti,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9997,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9998,Pucón,Chile,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9999,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3


In [53]:
#combined =  pd.concat([ds, df2["class_4"]])
combined = pd.concat([ds, df2], axis=1).reindex(ds.index)


In [54]:
combined = pd.concat([ds, df2], axis=1)

In [55]:
combined = combined[0:10000]
combined

,tweet_text,lang,timestamp,lat,lon,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
1,Nak berak masa berlari is another level of ann...,in,2021-08-24T00:20:01.000Z,3.148531,101.705305,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
2,@cricketaakash @vikrantgupta73 India never ha...,en,2021-08-19T07:27:34.000Z,30.730386,76.776524,India,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
3,,0,0,0.000000,0.000000,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
4,"Just posted a photo @ Cocoa,Stati Uniti,en,202...",0,0,0.000000,0.000000,Just,Stati Uniti,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
5,"كفيل في غضبي أكون اسوأ واحد..""",ar,2021-08-09T19:47:18.000Z,30.356045,31.331557,كفيل,0,0,0,"(کفیل, بخش مرکزی, شهرستان چایپاره, استان آذربا...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(38.9105051, 44.692649, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van Ngongo😃😃😃😃 https://t.co/AOcTfoQRCb,vi,2021-08-28T06:39:05.000Z,-13.247367,34.296137,Van,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9997,We see you #Knights! \n\nStory: https://t.co/1...,en,2021-08-18T15:35:35.000Z,33.194651,-96.699604,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9998,"Acaba de publicar una foto en Pucón,Chile,es,2...",0,0,0.000000,0.000000,Pucón,Chile,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9999,You don’t change a machine by voting out old p...,0,0,0.000000,0.000000,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3


In [56]:
combined

,tweet_text,lang,timestamp,lat,lon,0,1,2,3,location0,location1,location2,point0,point1,point2,dist1,dist2,dist3,class_4
1,Nak berak masa berlari is another level of ann...,in,2021-08-24T00:20:01.000Z,3.148531,101.705305,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
2,@cricketaakash @vikrantgupta73 India never ha...,en,2021-08-19T07:27:34.000Z,30.730386,76.776524,India,0,0,0,"(India, (22.3511148, 78.6677428))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(22.3511148, 78.6677428, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
3,,0,0,0.000000,0.000000,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
4,"Just posted a photo @ Cocoa,Stati Uniti,en,202...",0,0,0.000000,0.000000,Just,Stati Uniti,0,0,"(Just, Tęgoborze, gmina Łososina Dolna, powiat...","(United States, (39.7837304, -100.445882))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(49.7239543, 20.6463683, 0.0)","(39.7837304, -100.445882, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
5,"كفيل في غضبي أكون اسوأ واحد..""",ar,2021-08-09T19:47:18.000Z,30.356045,31.331557,كفيل,0,0,0,"(کفیل, بخش مرکزی, شهرستان چایپاره, استان آذربا...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(38.9105051, 44.692649, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,Van Ngongo😃😃😃😃 https://t.co/AOcTfoQRCb,vi,2021-08-28T06:39:05.000Z,-13.247367,34.296137,Van,0,0,0,"(Vannes, Morbihan, Bretagne, France métropolit...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(47.6586772, -2.7599079, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9997,We see you #Knights! \n\nStory: https://t.co/1...,en,2021-08-18T15:35:35.000Z,33.194651,-96.699604,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9998,"Acaba de publicar una foto en Pucón,Chile,es,2...",0,0,0.000000,0.000000,Pucón,Chile,0,0,"(Pucón, Provincia de Cautín, Región de la Arau...","(Chile, (-31.7613365, -71.3187697))","(None, Torino, Piemonte, Italia, (44.933143, 7...","(-39.273222, -71.9329968, 0.0)","(-31.7613365, -71.3187697, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3
9999,You don’t change a machine by voting out old p...,0,0,0.000000,0.000000,0,0,0,0,"(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(None, Torino, Piemonte, Italia, (44.933143, 7...","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)","(44.933143, 7.540121, 0.0)",4.014319e+06,5.032884e+06,5.032884e+06,3


In [57]:
l0 = ['tweet_text','class_4','location2']
l = ['tweet_text','class_4']

combined = combined[l]
comb = combined

In [58]:
combined

,tweet_text,class_4
1,Nak berak masa berlari is another level of ann...,3
2,@cricketaakash @vikrantgupta73 India never ha...,3
3,,3
4,"Just posted a photo @ Cocoa,Stati Uniti,en,202...",3
5,"كفيل في غضبي أكون اسوأ واحد..""",3
...,...,...
9996,Van Ngongo😃😃😃😃 https://t.co/AOcTfoQRCb,3
9997,We see you #Knights! \n\nStory: https://t.co/1...,3
9998,"Acaba de publicar una foto en Pucón,Chile,es,2...",3
9999,You don’t change a machine by voting out old p...,3


In [59]:
#ff = ff[l]
#ff

In [60]:
# BERT odel preparations

In [61]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [62]:
combined = combined.tweet_text.values

In [63]:
# Print the original tweet.

print(' Original: ', combined[0])

# Print the tweet split into tokens.

print('Tokenized: ', tokenizer.tokenize(combined[0]))

# Print the sentence mapped to token ID's.

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(combined[0])))

 Original:  Nak berak masa berlari is another level of annoying🥴
Tokenized:  ['nak', 'be', '##rak', 'mas', '##a', 'be', '##rl', '##ari', 'is', 'another', 'level', 'of', '[UNK]']
Token IDs:  [17823, 2022, 16555, 16137, 2050, 2022, 12190, 8486, 2003, 2178, 2504, 1997, 100]


In [64]:
max_len = 0

# For every sentence...

for text in combined:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Update the maximum sentence length.
    
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (745 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  895


In [65]:
# Making list of sentence lenghts:

token_lens = []

for text in combined:
    tokens = tokenizer.encode(text, max_length = 512)
    token_lens.append(len(tokens))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [66]:
# Splitting the train test data after tokenizing.
#labels = combined['class_4'].values
#idx = len(labels)
train= combined[:8000]
test = combined[8000:]
train.shape

(8000,)

In [67]:
def tokenize_map(sentence,labs='None'):
    
    """A function for tokenize all of the sentences and map the tokens to their word IDs."""
    
    global labels
    
    input_ids = []
    attention_masks = []

    # For every sentence...
    
    for text in sentence:
        #   "encode_plus" will:
        
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        
        encoded_dict = tokenizer.encode_plus(
                            text,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            truncation='longest_first', # Activate and control truncation
                            max_length = 400,           # Max length according to our text data.
                            pad_to_max_length = True, # Pad & truncate all sentences.
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the id list. 
        
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    if labs != 'None': # Setting this for using this definition for both train and test data so labels won't be a problem in our outputs.
        labels = torch.tensor(labels)
        return input_ids, attention_masks, labels
    else:
        return input_ids, attention_masks
            

In [68]:
labelst = df2['class_4'].values
labels = labelst[0:8000]


In [69]:
train = train.astype(str)
test = test.astype(str)
labels = labels.astype(int)

In [70]:
#labels = labels[0:10000]

In [71]:
type(labels)

numpy.ndarray

In [72]:

input_ids, attention_masks, labels = tokenize_map(train, labels)
test_input_ids, test_attention_masks= tokenize_map(test)

In [73]:
input_ids

tensor([[  101, 17823,  2022,  ...,     0,     0,     0],
        [  101,  1030,  4533,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0],
        ...,
        [  101,   100,  1811,  ...,     0,     0,     0],
        [  101,  6583,  2080,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0]])

In [74]:
attention_masks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [75]:
labels # just 4 class because all 1000 observation are too short

tensor([3, 3, 3,  ..., 3, 3, 3], dtype=torch.int32)

In [76]:
# Combine the training inputs into a TensorDataset.

dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

6,400 training samples
1,600 validation samples


In [77]:
# The DataLoader needs to know our batch size for training, so we specify it here. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.

batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [78]:
prediction_data = TensorDataset(test_input_ids, test_attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [79]:
# If there's a GPU available...

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.  
    
    device = torch.device('cuda')    


    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

No GPU available, using the CPU instead.


In [81]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained(
    'prajjwal1/bert-tiny', #nreimers/BERT-Tiny_L-2_H-128_A-2 https://huggingface.co/models?sort=downloads&search=tiny Use the 124-layer, 1024-hidden, 16-heads, 340M parameters BERT model with an uncased vocab.
    num_labels = 4, # CHANGE TO 4 in the CASE OF NEW TARGET NUBERS FROm 1 to 4 ASIDE FRO 4 only The number of output labels--2 for binary classification. You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the device which we set GPU in our case.

model.to(device)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, element

In [82]:
# or this model

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(5, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [83]:
# Get all of the model's parameters as a list of tuples:

params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print('{:<55} {:>12}'.format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print('{:<55} {:>12}'.format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print('{:<55} {:>12}'.format(p[0], str(tuple(p[1].size()))))


The BERT model has 41 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 128)
bert.embeddings.position_embeddings.weight                (512, 128)
bert.embeddings.token_type_embeddings.weight                (2, 128)
bert.embeddings.LayerNorm.weight                              (128,)
bert.embeddings.LayerNorm.bias                                (128,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (128, 128)
bert.encoder.layer.0.attention.self.query.bias                (128,)
bert.encoder.layer.0.attention.self.key.weight            (128, 128)
bert.encoder.layer.0.attention.self.key.bias                  (128,)
bert.encoder.layer.0.attention.self.value.weight          (128, 128)
bert.encoder.layer.0.attention.self.value.bias                (128,)
bert.encoder.layer.0.attention.output.dense.weight        (128, 128)
bert.encoder.layer.0.attention.output.dense.bias              (1

In [84]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch).

# The 'W' stands for 'Weight Decay fix' probably...

optimizer = AdamW(model.parameters(),
                  lr = 6e-6, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon
                )

In [85]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 

# We chose to run for 3, but we'll see later that this may be over-fitting the training data.

epochs = 2

# Total number of training steps is [number of batches] x [number of epochs] (Note that this is not the same as the number of training samples).
total_steps = (len(train_dataloader) * epochs)

# Create the learning rate scheduler.

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [86]:
def flat_accuracy(preds, labels):
    
    """A function for calculating accuracy scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return accuracy_score(labels_flat, pred_flat)

def flat_f1(preds, labels):
    
    """A function for calculating f1 scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return f1_score(labels_flat, pred_flat)

In [87]:
def format_time(elapsed):    
    
    """A function that takes a time in seconds and returns a string hh:mm:ss"""
    
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [96]:
# second variant for non working in the case of attribute ite error

training_stats = []

# Measure the total training time for the whole run.

total_t0 = time.time()

# For each epoch...

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print('')
    print()
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes:
    
    t0 = time.time()

    # Reset the total loss for this epoch.
    
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to `train` just changes the *mode*, it doesn't *perform* the training.
    
    # `dropout` and `batchnorm` layers behave differently during training vs. test ,
    # source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
    
    model.train()

    # For each batch of training data...
    
    for step, batch in enumerate(train_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the device(gpu in our case) using the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        
        b_input_ids = batch[0].to(device).to(torch.int64)
        b_input_mask = batch[1].to(device).to(torch.int64)
        b_labels = batch[2].to(device).to(torch.int64)

        # Always clear any previously calculated gradients before performing a backward pass. PyTorch doesn't do this automatically because accumulating the gradients is 'convenient while training RNNs'. 
        # Source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is down here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers BertForSequenceClassification.
        
        # It returns different numbers of parameters depending on what arguments given and what flags are set. For our useage here, it returns the loss (because we provided labels),
        # And the 'logits' (the model outputs prior to activation.)
        
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can calculate the average loss at the end, 
        # `loss` is a tensor containing a single value; the `.item()` function just returns the Python value from the tensor.
        
        #total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        
        #loss.backward()

        # Clip the norm of the gradients to 1.0 This is to help prevent the 'exploding gradients' problem.
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        
        # The optimizer dictates the 'update rule'(How the parameters are modified based on their gradients, the learning rate, etc.)
        
        optimizer.step()

        # Update the learning rate.
        
        scheduler.step()

    # Calculate the average loss over all of the batches.
    
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    
    training_time = format_time(time.time() - t0)

    print('')
    print('  Average training loss: {0:.2f}'.format(avg_train_loss))
    print('  Training epcoh took: {:}'.format(training_time))
        




======== Epoch 1 / 2 ========
Training...
  Batch    50  of    200.    Elapsed: 0:00:28.
  Batch   100  of    200.    Elapsed: 0:00:55.
  Batch   150  of    200.    Elapsed: 0:01:22.

  Average training loss: 0.00
  Training epcoh took: 0:01:50


======== Epoch 2 / 2 ========
Training...
  Batch    50  of    200.    Elapsed: 0:00:28.
  Batch   100  of    200.    Elapsed: 0:00:55.
  Batch   150  of    200.    Elapsed: 0:01:23.

  Average training loss: 0.00
  Training epcoh took: 0:01:50


In [ ]:
#train = Yt_train.type(torch.LongTensor)

In [97]:
# Prediction on test set:

print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

# Put model in evaluation mode:

model.eval()

# Tracking variables :

predictions = []

# Predict:

for batch in prediction_dataloader:
    
  # Add batch to GPU

    batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader:
    
    b_input_ids, b_input_mask, = batch
  
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction:

    with torch.no_grad():
      # Forward pass, calculate logit predictions:
    
          outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

  # Move logits and labels to CPU:
    
    logits = logits.detach().cpu().numpy()
 
  
  # Store predictions and true labels:
    
    predictions.append(logits)


print('    DONE.')

Predicting labels for 2,000 test sentences...
    DONE.


In [98]:
# Getting list of predictions and then choosing the target value with using argmax on probabilities.

flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [99]:
combined

array(['Nak berak masa berlari is another level of annoying🥴',
       "@cricketaakash @vikrantgupta73  India never had depth in bowling resources before the current setup??\n2003-08 era: Kumble, Agarkar, Nehra, Zaheer, Balaji\nWe probably didn't manage/use them well",
       '  ', ...,
       'Acaba de publicar una foto en Pucón,Chile,es,2021-08-11T02:54:15.000Z,-39.29045965\n@MarDGamero @J_Zaragoza_ Pues cómo se llega a Presidente del Gobierno el mayor inútil de España...!!!!!,es,2021-08-24T17:04:57.000Z,36.66400575,-4.57842605\nGovernment is a machine.',
       'You don’t change a machine by voting out old parts for new parts.',
       'Machine will still run the same.'], dtype=object)

In [100]:
submission = comb[8000:10000]
submission['target'] = flat_predictions
submission.head(10)

,tweet_text,class_4,target
8001,@felipepeixotoap Kkkkkkk,3,0
8002,É bloqueio do Brasil BB 😍😍😍😍,3,3
8003,"I’m not even going to hold you,Etats-Unis,en,2...",3,1
8004,#InstaGood,3,0
8005,#VivirSinComplicarmeLaVida,3,1
8006,#SueñaBonitoSeCumple,3,1
8007,#LoveWins en Mexico City,3,0
8008,Spahn: Deutschland spendet ab sofort Impfstoff...,3,3
8009,Acaba de publicar un video en San Ignacio De S...,3,1
8010,"Just posted a photo @ Houston,Etats-Unis,en,20...",3,3


In [101]:
y_true = submission['class_4'].to_numpy().astype(float)
y_pred = submission['target'].to_numpy().astype(float)

In [102]:
y_true

array([3., 3., 3., ..., 3., 3., 3.])

In [103]:
f1_score(y_true, y_pred, average=None)

array([0.        , 0.        , 0.44479005])

In [104]:
f1_score(y_true, y_pred, average='macro')

0.14826334888543286

In [105]:
f1_score(y_true, y_pred, average='weighted')

0.44479004665629857

In [106]:
f1_score(y_true, y_pred, average='micro')

0.286

In [ ]:
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.

    print('')
    print('Running Validation...')

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    
    model.eval()

    # Tracking variables:
    
    total_eval_accuracy = 0
    total_eval_loss = 0
    total_eval_f1 = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch.
    
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        
        # As we unpack the batch, we'll also copy each tensor to the GPU using the `to` method.
        
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during the forward pass, since this is only needed for backprop (training part).
        
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the 'segment ids', which differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is down here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers BertForSequenceClassification.
            # Get the 'logits' output by the model. The 'logits' are the output values prior to applying an activation function like the softmax.
            
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        
        #total_eval_loss += loss.item()

        # Move logits and labels to CPU:
        
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and accumulate it over all batches:
        
        #total_eval_accuracy += flat_accuracy(logits, label_ids)
        #total_eval_f1 += flat_f1(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    
    #avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    #print('  Accuracy: {0:.2f}'.format(avg_val_accuracy))
    
    # Report the final f1 score for this validation run.
    
    #avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    #print('  F1: {0:.2f}'.format(avg_val_f1))

    # Calculate the average loss over all of the batches.
    
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    
    
    # Measure how long the validation run took:
    
    validation_time = format_time(time.time() - t0)
    
    print('  Validation Loss: {0:.2f}'.format(avg_val_loss))
    print('  Validation took: {:}'.format(validation_time))

    # Record all statistics from this epoch.
    
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            #'Valid. Accur.': avg_val_accuracy,
            #'Val_F1' : avg_val_f1,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print('')
print('Training complete!')

print('Total training took {:} (h:mm:ss)'.format(format_time(time.time()-total_t0)))